In [1]:
# load relevant packages
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import datetime

from sklearn import tree

from gurobipy import *

In [2]:
# load the data
file_path = '/Users/ericjiang/Desktop/msci719_assignment/ass7/'
file_name = 'data_2.csv'
df = pd.read_csv(file_path + file_name)

In [3]:
df.head()

,Item#,Price,Weekend,Event_Length,Branded,Color_Popularity,ConcurrentEvents,Number_Competing_Styles_in_Event,Relative_Price_of_Competing_Styles,Num_Branded_Events12,Brand_MSRP_Index
0,1,129.9,1,2,0,0.31,1,52,1.65,0,1.20
1,2,59.9,0,3,1,0.08,4,56,1.53,5,0.87
2,3,39.9,1,3,1,0.31,2,47,0.80,1,0.92
3,4,59.9,0,2,1,0.08,3,12,1.24,6,1.18
4,5,79.9,0,2,1,0.31,2,13,0.98,5,1.04


In [4]:
# load the file with true demand of each item
demand_file = 'result.csv'
df_demand = pd.read_csv(demand_file)

In [5]:
df_demand.head()

,Item#,Demand
0,1,91.0
1,2,80.0
2,3,91.0
3,4,171.0
4,5,85.0


In [6]:
# merge the two datasets
df = df.merge(df_demand,how='inner',left_on='Item#',right_on='Item#')

In [7]:
df.head()

,Item#,Price,Weekend,Event_Length,Branded,Color_Popularity,ConcurrentEvents,Number_Competing_Styles_in_Event,Relative_Price_of_Competing_Styles,Num_Branded_Events12,Brand_MSRP_Index,Demand
0,1,129.9,1,2,0,0.31,1,52,1.65,0,1.20,91.0
1,2,59.9,0,3,1,0.08,4,56,1.53,5,0.87,80.0
2,3,39.9,1,3,1,0.31,2,47,0.80,1,0.92,91.0
3,4,59.9,0,2,1,0.08,3,12,1.24,6,1.18,171.0
4,5,79.9,0,2,1,0.31,2,13,0.98,5,1.04,85.0


In [8]:
# train a regression tree model with max_depth=10 (to avoid overfitting)
clf = tree.DecisionTreeRegressor(max_depth=10)
X = df.drop(['Demand','Item#'],axis=1)
y = df['Demand']
clf = clf.fit(X, y)

In [9]:
# load the file with first-exposure items
test_file = 'First-exposures.csv'
df_test = pd.read_csv(test_file)

In [10]:
df_test

,item#,Weekend,Event_Length,Branded,Color_Popularity,ConcurrentEvents,Number_Competing_Styles_in_Event,Num_Branded_Events12,Brand_MSRP_Index,Event
0,A,1,2,1,0.17,2,3,2,0.87,A
1,B,0,1,0,0.65,4,2,1,0.92,B
2,C,1,2,1,0.08,2,3,1,1.18,A
3,D,1,2,1,0.75,2,3,0,0.46,A
4,E,1,1,0,0.44,4,2,5,2.18,B


In [11]:
# setup the optimization model for further use
milp_model = Model('milp')
milp_model.setParam('OutputFlag', 0)

Restricted license - for non-production use only - expires 2022-01-13


In [12]:
# create the price list for event A and the possible k values
price_list_A = [25.99,30.99,35.99,40.99]
k_A = np.linspace(3*price_list_A[0],3*price_list_A[3],10)

In [13]:
k_A

array([ 77.97,  82.97,  87.97,  92.97,  97.97, 102.97, 107.97, 112.97,
       117.97, 122.97])

In [14]:
# extract the items regarding item A
df_test_A = df_test[df_test['Event']=='A'].reset_index().drop('index',axis=1)
df_test_A = df_test_A.drop(['item#','Event'],axis=1)

In [15]:
df_test_A

,Weekend,Event_Length,Branded,Color_Popularity,ConcurrentEvents,Number_Competing_Styles_in_Event,Num_Branded_Events12,Brand_MSRP_Index
0,1,2,1,0.17,2,3,2,0.87
1,1,2,1,0.08,2,3,1,1.18
2,1,2,1,0.75,2,3,0,0.46


In [16]:
# define the function to calculate relative price
def get_relative_price(price,k,n):
    return price/(k/n)

In [17]:
####### main iteration part for event A #######
for k in list(k_A):
    # store the demand in each possible scenario
    demand_all = []
    for i in range(len(df_test_A)):
        # format and feed the data into the trained model to predict demand
        demand = []
        for price in price_list_A:
            predictor = []
            predictor.append(price) 
            predictor.extend(list(df_test_A.iloc[i][:6]))
            predictor.append(get_relative_price(price,k,3))
            predictor.extend(list(df_test_A.iloc[i][6:]))
            demand.append(float(clf.predict([predictor])))
        demand_all.append(demand)
        
    #### optimization part ####
    # define decision variables
    x11 = milp_model.addVar(vtype=GRB.BINARY, name="x11")
    x12 = milp_model.addVar(vtype=GRB.BINARY, name="x12")
    x13 = milp_model.addVar(vtype=GRB.BINARY, name="x13")
    x14 = milp_model.addVar(vtype=GRB.BINARY, name="x14")
    x21 = milp_model.addVar(vtype=GRB.BINARY, name="x21")
    x22 = milp_model.addVar(vtype=GRB.BINARY, name="x22")
    x23 = milp_model.addVar(vtype=GRB.BINARY, name="x23")
    x24 = milp_model.addVar(vtype=GRB.BINARY, name="x24")
    x31 = milp_model.addVar(vtype=GRB.BINARY, name="x31")
    x32 = milp_model.addVar(vtype=GRB.BINARY, name="x32")
    x33 = milp_model.addVar(vtype=GRB.BINARY, name="x33")
    x34 = milp_model.addVar(vtype=GRB.BINARY, name="x34")
    
    # get the price for each scenario
    p11, p12, p13, p14 = price_list_A[0], price_list_A[1], price_list_A[2], price_list_A[3]
    p21, p22, p23, p24 = price_list_A[0], price_list_A[1], price_list_A[2], price_list_A[3]
    p31, p32, p33, p34 = price_list_A[0], price_list_A[1], price_list_A[2], price_list_A[3]
    
    # get the demand for each scenario
    d11, d12, d13, d14 = demand_all[0][0], demand_all[0][1], demand_all[0][2], demand_all[0][3]
    d21, d22, d23, d24 = demand_all[1][0], demand_all[1][1], demand_all[1][2], demand_all[1][3]
    d31, d32, d33, d34 = demand_all[2][0], demand_all[2][1], demand_all[2][2], demand_all[2][3]
    
    # define the objective function
    obj_fn = x11*p11*d11 + x12*p12*d12 + x13*p13*d13 + x14*p14*d14 +\
             x21*p21*d21 + x22*p22*d22 + x23*p23*d23 + x24*p24*d24 +\
             x31*p31*d31 + x32*p32*d32 + x33*p33*d33 + x34*p34*d34
    
    milp_model.setObjective(obj_fn, GRB.MAXIMIZE)
    
    # add constraints
    milp_model.addConstr(x11 + x12 + x13 + x14 == 1, "c0")
    milp_model.addConstr(x21 + x22 + x23 + x24 == 1, "c1")
    milp_model.addConstr(x31 + x32 + x33 + x34 == 1, "c2")
    milp_model.addConstr(x11*p11 + x12*p12 + x13*p13 + x14*p14 +\
                         x21*p21 + x22*p22 + x23*p23 + x24*p24 +\
                         x31*p31 + x32*p32 + x33*p33 + x34*p34 == k, "c3")
    
    # solve the optimization problem
    milp_model.optimize()
    print('when k='+str(k)+':')
    print('Objective Function Value: %.2f' % milp_model.objVal)
    for v in milp_model.getVars()[-12:]:
        print('%s: %g' % (v.varName, v.x))
    print()
    print()  

when k=77.97:
Objective Function Value: 12339.52
x11: 1
x12: -0
x13: -0
x14: -0
x21: 1
x22: -0
x23: -0
x24: -0
x31: 1
x32: -0
x33: -0
x34: -0


when k=82.97:
Objective Function Value: 11299.20
x11: 1
x12: 0
x13: -0
x14: -0
x21: -0
x22: 1
x23: -0
x24: -0
x31: 1
x32: -0
x33: -0
x34: -0


when k=87.97:
Objective Function Value: 14233.42
x11: -0
x12: 1
x13: -0
x14: -0
x21: -0
x22: 1
x23: -0
x24: 0
x31: 1
x32: -0
x33: -0
x34: -0


when k=92.97:
Objective Function Value: 14713.42
x11: -0
x12: 1
x13: -0
x14: -0
x21: -0
x22: 1
x23: -0
x24: -0
x31: -0
x32: 1
x33: -0
x34: -0


when k=97.97:
Objective Function Value: 15243.12
x11: -0
x12: -0
x13: -0
x14: 1
x21: -0
x22: 1
x23: -0
x24: -0
x31: 1
x32: -0
x33: -0
x34: -0


when k=102.97:
Objective Function Value: 14213.08
x11: -0
x12: -0
x13: 1
x14: -0
x21: -0
x22: -0
x23: -0
x24: 1
x31: 1
x32: -0
x33: -0
x34: -0


when k=107.97:
Objective Function Value: 14885.37
x11: -0
x12: -0
x13: -0
x14: 1
x21: -0
x22: -0
x23: -0
x24: 1
x31: 1
x32: -0
x33: -0
x3

In [18]:
# create the price list for event B and the possible k values
price_list_B = [45.99,50.99]
k_B = np.linspace(2*price_list_B[0],2*price_list_B[1],3)

In [19]:
k_B

array([ 91.98,  96.98, 101.98])

In [20]:
# extract the items regarding item B
df_test_B = df_test[df_test['Event']=='B'].reset_index().drop('index',axis=1)
df_test_B = df_test_B.drop(['item#','Event'],axis=1)

In [21]:
df_test_B

,Weekend,Event_Length,Branded,Color_Popularity,ConcurrentEvents,Number_Competing_Styles_in_Event,Num_Branded_Events12,Brand_MSRP_Index
0,0,1,0,0.65,4,2,1,0.92
1,1,1,0,0.44,4,2,5,2.18


In [22]:
####### main iteration part for event B #######
for k in list(k_B):
    # store the demand in each possible scenario
    demand_all = []
    for i in range(len(df_test_B)):
        # format and feed the data into the trained model to predict demand
        demand = []
        for price in price_list_B:
            predictor = []
            predictor.append(price) 
            predictor.extend(list(df_test_B.iloc[i][:6]))
            predictor.append(get_relative_price(price,k,2))
            predictor.extend(list(df_test_A.iloc[i][6:]))
            demand.append(float(clf.predict([predictor])))
        demand_all.append(demand)
        
    #### optimization part ####
    # define decision variables
    x11 = milp_model.addVar(vtype=GRB.BINARY, name="x11")
    x12 = milp_model.addVar(vtype=GRB.BINARY, name="x12")
    x21 = milp_model.addVar(vtype=GRB.BINARY, name="x21")
    x22 = milp_model.addVar(vtype=GRB.BINARY, name="x22")
    
    # get the price for each scenario
    p11, p12 = price_list_B[0], price_list_B[1]
    p21, p22 = price_list_B[0], price_list_B[1]
    
    # get the demand for each scenario
    d11, d12 = demand_all[0][0], demand_all[0][1]
    d21, d22 = demand_all[1][0], demand_all[1][1]
    
    # define the objective function
    obj_fn = x11*p11*d11 + x12*p12*d12 +\
             x21*p21*d21 + x22*p22*d22
    
    milp_model.setObjective(obj_fn, GRB.MAXIMIZE)
    
    # add constraints
    milp_model.addConstr(x11 + x12 == 1, "c0")
    milp_model.addConstr(x21 + x22 == 1, "c1")
    milp_model.addConstr(x11*p11 + x12*p12 +\
                         x21*p21 + x22*p22 == k, "c2")
    
    # solve the optimization problem
    milp_model.optimize()
    print('when k='+str(k)+':')
    print('Objective Function Value: %.2f' % milp_model.objVal)
    for v in milp_model.getVars()[-4:]:
        print('%s: %g' % (v.varName, v.x))
    print()
    print()

when k=91.98:
Objective Function Value: 9548.87
x11: 1
x12: 0
x21: 1
x22: 0


when k=96.98:
Objective Function Value: 9618.49
x11: 1
x12: 0
x21: -0
x22: 1


when k=101.98:
Objective Function Value: 10587.02
x11: 0
x12: 1
x21: 0
x22: 1


